In [11]:
# notebooks/review_highlights.ipynb

import os, glob, json, webbrowser
from pathlib import Path
from IPython.display import HTML, display

# ---------------------------
# 0. Set project root
# ---------------------------
PROJECT_ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd()
os.chdir(PROJECT_ROOT)
print(f"📂 Working directory set to: {PROJECT_ROOT}")

# ---------------------------
# Helpers
# ---------------------------
def show_video(path, width=400):
    """Embed video inline (muted in VS Code/Jupyter)."""
    path = Path(path).as_posix()  # forward slashes
    return HTML(f"""
    <video width="{width}" controls>
        <source src="{path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """)

def open_in_browser(path):
    """Open video file in system browser (with sound)."""
    path = Path(path).resolve()
    url = f"file://{path.as_posix()}"
    webbrowser.open(url)
    print(f"🌐 Opened in browser: {url}")

# ---------------------------
# 1. Load the latest run log
# ---------------------------
Path("runs").mkdir(exist_ok=True)
logs = sorted(glob.glob("runs/*.json"), key=os.path.getmtime, reverse=True)

if not logs:
    # If no logs exist, create a dummy one
    dummy_log = {
        "clips": [
            {
                "rank": 1,
                "score": 0.95,
                "text": "Sample transcript for clip 1.",
                "file": "data/clips/clip_1.mp4"
            }
        ]
    }
    dummy_log_path = "runs/dummy_run.json"
    with open(dummy_log_path, "w", encoding="utf-8") as f:
        json.dump(dummy_log, f, indent=2)
    logs = [dummy_log_path]

latest_log = logs[0]
print(f"📂 Loaded run: {latest_log}")

with open(latest_log, "r", encoding="utf-8") as f:
    run_data = json.load(f)

clips = run_data.get("clips", [])

# ---------------------------
# 2. Prepare reviews folder
# ---------------------------
Path("notebooks/reviews").mkdir(parents=True, exist_ok=True)
review_file = f"notebooks/reviews/reviewed_{os.path.basename(latest_log)}"

# ---------------------------
# 3. Review loop
# ---------------------------
feedback = []
for clip in clips:
    print(f"\n🎬 Clip {clip['rank']} | Score={clip['score']:.3f}")
    print(f"📝 {clip['text'][:200]}...")  # transcript preview

    # Ensure valid video path
    rel_path = Path(clip.get("file", ""))
    file_path = PROJECT_ROOT / rel_path
    if file_path.is_file() and file_path.suffix.lower() in [".mp4", ".mov", ".avi"]:
        # Inline (preview only, may be muted)
        display(show_video(file_path, width=400))
        # Auto-open in browser with sound
        open_in_browser(file_path)
    else:
        print(f"⚠️ No valid video file for this clip: {file_path}")

    # Ask for feedback interactively
    val = input("Highlight? 👍(1) / 👎(0): ")
    val = 1 if val.strip() == "1" else 0

    clip["label"] = val
    feedback.append(clip)

# ---------------------------
# 4. Save feedback
# ---------------------------
with open(review_file, "w", encoding="utf-8") as f:
    json.dump({"original_run": latest_log, "feedback": feedback}, f, indent=2)

print(f"✅ Saved feedback to {review_file}")


📂 Working directory set to: /Users/varunreddy/Downloads/Reels
📂 Loaded run: runs/2025-08-28_21-00-28.json

🎬 Clip 1 | Score=0.412
📝 Are you the best footballer in the world right now? I think so, yes. In my mind I'm always the best. I don't care what the people are thinking, what they say. In my mind, not just this year but always...


🌐 Opened in browser: file:///Users/varunreddy/Downloads/Reels/data/clips/clip_1.mp4

🎬 Clip 2 | Score=0.410
📝 Maybe it's someone's, it doesn't like me because maybe I'm too good. I don't follow the record. The records follow me in my mind. I'm always good. The talent is part of me, but if you don't work the t...


🌐 Opened in browser: file:///Users/varunreddy/Downloads/Reels/data/clips/clip_2.mp4

🎬 Clip 3 | Score=0.405
📝 But they have to say to be in my shoes is not easy but I'm not complaining. I just want to say that I'm feel happy. All this happened because the reason. The reason is I'm unbelievable inside the pitc...


🌐 Opened in browser: file:///Users/varunreddy/Downloads/Reels/data/clips/clip_3.mp4

🎬 Clip 4 | Score=0.402
📝 How much has Leonel Messi pushed you to become a better player? I'm pushed to myself. I don't need other players to push myself. I just want to see that I was in sporting. I push myself all the time t...


🌐 Opened in browser: file:///Users/varunreddy/Downloads/Reels/data/clips/clip_4.mp4

🎬 Clip 5 | Score=0.402
📝 How much has Leonel Messi pushed you to become a better player? I'm pushed to myself. I don't need other players to push myself. I just want to see that I was in sporting. I push myself all the time t...


🌐 Opened in browser: file:///Users/varunreddy/Downloads/Reels/data/clips/clip_5.mp4
✅ Saved feedback to notebooks/reviews/reviewed_2025-08-28_21-00-28.json
